In [1]:
suppressMessages(library(Signac))
suppressMessages(library(Seurat))
suppressMessages(library(EnsDb.Hsapiens.v86))
suppressMessages(library(BSgenome.Hsapiens.UCSC.hg38))
suppressMessages(library(JASPAR2020))
suppressMessages(library(TFBSTools))

set.seed(1234)

In [2]:
# load the RNA and ATAC data
counts <- Read10X_h5("./pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.h5")

Genome matrix has multiple modalities, returning a list of matrices for this genome



In [3]:
# create a Seurat object containing the RNA adata
pbmc <- CreateSeuratObject(
  counts = counts$`Gene Expression`,
  assay = "RNA"
)

In [4]:
peaks <- counts$Peaks

In [9]:
# clean up the peaks
peaks <- peaks[!grepl("chrM", rownames(peaks)), ]
peaks <- peaks[!grepl("random", rownames(peaks)), ]
peaks <- peaks[!grepl("alt", rownames(peaks)), ]
peaks <- peaks[!grepl("KI", rownames(peaks)), ]
peaks <- peaks[!grepl("GL", rownames(peaks)), ]

In [10]:
# create ATAC assay and add it to the object
pbmc[["ATAC"]] <- CreateChromatinAssay(
  counts = peaks,
  sep = c(":", "-")
)

In [11]:
pbmc@assays$ATAC@ranges

GRanges object with 98290 ranges and 0 metadata columns:
          seqnames              ranges strand
             <Rle>           <IRanges>  <Rle>
      [1]     chr1          9768-10660      *
      [2]     chr1       180582-181297      *
      [3]     chr1       181404-181887      *
      [4]     chr1       191175-192089      *
      [5]     chr1       267561-268455      *
      ...      ...                 ...    ...
  [98286]     chrX 156030114-156030945      *
  [98287]     chrY   11295202-11296181      *
  [98288]     chrY   11332853-11333545      *
  [98289]     chrY   11333667-11334374      *
  [98290]     chrY   56836234-56837094      *
  -------
  seqinfo: 24 sequences from an unspecified genome; no seqlengths

In [12]:
DefaultAssay(pbmc) <- "ATAC"

In [13]:
# Get a list of motif position frequency matrices from the JASPAR database
pfm <- getMatrixSet(
  x = JASPAR2020,
  opts = list(collection = "CORE", tax_group = 'vertebrates', all_versions = FALSE)
)

# add motif information
pbmc <- AddMotifs(
  object = pbmc,
  genome = BSgenome.Hsapiens.UCSC.hg38,
  pfm = pfm
)

Building motif matrix

Finding motif positions

Creating Motif object



In [14]:
pbmc <- RunChromVAR(
  object = pbmc,
  genome = BSgenome.Hsapiens.UCSC.hg38
)

Computing GC bias per region

Selecting background regions

Computing deviations from background

Constructing chromVAR assay



In [15]:
pbmc

An object of class Seurat 
135637 features across 2711 samples within 3 assays 
Active assay: ATAC (98290 features, 0 variable features)
 2 other assays present: RNA, chromvar